In [ ]:
from openai import OpenAI, APIError, RateLimitError
from google.colab import userdata
from collections import Counter
import re
import time

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("'OPENAI_API_KEY' 이름의 보안 비밀이 설정되지 않았습니다.")
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def extract_seat_recommendation(text: str) -> str | None:
    """텍스트에서 'X열 Y위치' 형태의 좌석 추천 추출"""
    match = re.search(r"([A-Z]+열\s+[가-힣]+(?:쪽|블록)?)", text)
    if match:
        seat = re.sub(r"(?:쪽|블록)", "", match.group(1)).strip()
        return seat

    match = re.search(r"(?:추천|좌석은?):\s*([A-Z]+열\s+[가-힣]+)", text)
    if match:
        return match.group(1)

    match = re.search(r"([A-Z]+열)\s+([가-힣]+)", text)
    if match:
        return f"{match.group(1)} {match.group(2)}"

    return None

In [ ]:
def get_consistent_movie_seat(prompt: str, num_samples: int = 7) -> str:
    """영화 좌석 추천에 Self-Consistency 적용"""

    print(f"\nSelf-Consistency 실행 (샘플 수: {num_samples})")
    system_message = "당신은 최고의 영화 관람 경험을 위한 좌석 추천 전문가입니다. 시야각, 음향 효과 등을 고려하여 단계별로 생각하고, 최종 추천 좌석(예: 'D열 중앙')을 명확히 언급해주세요."

    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.8,  # 약간 높은 온도로 다양한 추천 유도
                n=num_samples,
            )
            break
        except (APIError, RateLimitError) as e:
            print(
                f"API 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}. 잠시 후 재시도합니다..."
            )
            if attempt == max_retries - 1:
                raise
            time.sleep(2**attempt)

    recommendations: list[str] = []
    print("\n--- 생성된 추천 및 추출된 좌석 ---")
    for i, choice in enumerate(completion.choices):
        response_text = choice.message.content.replace("\n", " ")
        seat = extract_seat_recommendation(response_text)
        print(f"샘플 {i + 1}: {response_text[:60]}... => 추출된 좌석: {seat}")
        if seat:
            recommendations.append(seat)

    if not recommendations:
        return "유효한 좌석 추천을 추출하지 못했습니다."

    # 합의 도출 (Majority Vote)
    seat_counts = Counter(recommendations)
    most_common_seat, count = seat_counts.most_common(1)[0]
    print(f"\n--- 집계 결과 ---")
    print(f"추천된 좌석 빈도: {dict(seat_counts)}")
    print(f"최종 선택된 좌석: {most_common_seat} ({count}/{len(recommendations)} 회)")

    return most_common_seat

In [ ]:
# 실행 및 결과 확인
prompt = "영화 관람 경험을 극대화하려면 어떤 좌석이 가장 좋을까요? 시야각과 음향 효과를 고려해서 추천해주세요."
consistent_seat = get_consistent_movie_seat(prompt, num_samples=7)
print(f"\n김주현 개발자가 얻은 최종 일관된 추천 좌석: {consistent_seat}")


Self-Consistency 실행 (샘플 수: 7)

--- 생성된 추천 및 추출된 좌석 ---
샘플 1: 영화 관람 경험을 극대화하기 위한 최적의 좌석을 추천하기 위해 몇 가지 단계로 나누어 생각해보겠습니다.  1... => 추출된 좌석: F열 사이에
샘플 2: 영화 관람 경험을 극대화하기 위해서는 시야각과 음향 효과가 모두 고려된 좌석을 선택하는 것이 중요합니다. 다... => 추출된 좌석: D열 중앙
샘플 3: 영화 관람 경험을 극대화하기 위해서는 시야각과 음향 효과를 함께 고려해야 합니다. 다음은 최적의 좌석을 선택... => 추출된 좌석: H열 중앙
샘플 4: 최고의 영화 관람 경험을 위해 좌석을 선택할 때, 시야각과 음향 효과를 고려하는 것이 매우 중요합니다. 다음... => 추출된 좌석: D열 또는
샘플 5: 영화 관람 경험을 극대화하기 위해서는 몇 가지 중요한 요소를 고려해야 합니다. 시야각과 음향 효과는 그 중에... => 추출된 좌석: D열 중앙
샘플 6: 영화 관람 경험을 극대화하기 위해서는 여러 요소를 고려해야 합니다. 주요 고려사항은 시야각, 음향 효과, 그... => 추출된 좌석: D열 중앙
샘플 7: 영화 관람 경험을 극대화하기 위해서는 몇 가지 요소를 고려해야 합니다. 아래의 단계에 따라 최적의 좌석을 추... => 추출된 좌석: D열 중앙

--- 집계 결과 ---
추천된 좌석 빈도: {'F열 사이에': 1, 'D열 중앙': 4, 'H열 중앙': 1, 'D열 또는': 1}
최종 선택된 좌석: D열 중앙 (4/7 회)

김주현 개발자가 얻은 최종 일관된 추천 좌석: D열 중앙
